# Volume Profile



In [ ]:
# ticker = "SPY"
interval = "5m"
previous_days = 5
resolution_usd = 0.1



## Package management



In [ ]:
import datetime
import functools
import math

from typing import List, Tuple, Union



In [ ]:
Version = Union[List[int], Tuple]



In [ ]:
def check_version(req:Version, version_str:str):
  v0, v1, v2 = version_str.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



How do I unload (reload) a Python module? https://stackoverflow.com/questions/437589/how-do-i-unload-reload-a-python-module



In [ ]:
import pandas as pd
if not check_version([1, 3, 5], pd.__version__):
  !pip install --upgrade pandas
  import importlib
  importlib.reload(pd)
  assert check_version([1, 3, 5], pd.__version__)



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

Download Financial Dataset Using Yahoo Finance in Python | A Complete Guide https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/



yfinance Library – A Complete Guide https://algotrading101.com/learn/yfinance-guide/


In [ ]:
import pandas as pd



In [ ]:
try:
  import yfinance as yf
except ModuleNotFoundError:
  !pip install yfinance
  !pip install yahoofinancials
  import yfinance as yf



Charting Candlestick_OHLC one minute bars with Pandas and Matplotlib https://stackoverflow.com/questions/41821916/charting-candlestick-ohlc-one-minute-bars-with-pandas-and-matplotlib

Candlestick Chart in Python (mplfinance, plotly, bokeh, bqplot and cufflinks) https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh#1

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
import scipy.signal as signal



## Data period

In [ ]:
@functools.lru_cache()
def tz() -> str:
  return "America/New_York"



Most recent previous business day in Python https://stackoverflow.com/questions/2224742/most-recent-previous-business-day-in-python and https://stackoverflow.com/a/51528258



In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
@functools.lru_cache()
def USBday():
  return pd.tseries.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())



pandas.Series.dt.tz_convert https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.tz_convert.html

In [ ]:
@functools.lru_cache()
def today(tz=tz()) -> pd.Timestamp.date:
  return pd.Timestamp.today(tz=tz).date()



In [ ]:
@functools.lru_cache()
def get_start_date(n_days:int, end_date=today(), unit=USBday()):
  return end_date + unit * (-n_days)



Python for Finance, Part I: Yahoo & Google Finance API, pandas, and matplotlib
https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

An Even Easier Guide to Getting Stock Data With Python
https://towardsdatascience.com/an-even-easier-guide-to-getting-stock-data-with-python-1a109df6b593



Google Finance API and Its Alternatives (Updated for 2021) https://rapidapi.com/blog/google-finance-api-alternatives/

## Getting the data



In [ ]:
@functools.lru_cache()
def get_df_start_end(ticker:str, start:pd.Timestamp, end:pd.Timestamp=today(), interval:str=interval):
  tic = yf.Ticker(ticker)
  return tic.history(start=start, end=end, interval=interval)



In [ ]:
def get_df_one_day(ticker:str, date:pd.Timestamp.date, interval:str=interval) -> pd.DataFrame:
  start = pd.Timestamp(date, unit="s")
  end = pd.Timestamp(date+pd.Timedelta(1, 'D'), unit="s") + pd.Timedelta(-1, 's')
  return get_df_start_end(ticker=ticker, start=start, end=end, interval=interval)



In [ ]:
def get_df(ticker:str, n_days:int, end_date:pd.Timestamp=today(), interval:str=interval):
  return get_df_start_end(ticker, get_start_date(n_days, end_date=end_date), end=end_date, interval=interval)



## Plotting the chart



How can I customize mplfinance.plot? https://stackoverflow.com/questions/60599812/how-can-i-customize-mplfinance-plot

Matplotib Finance (mplfinance) formatting axes of chart unsing mpf.plot() https://stackoverflow.com/questions/67232404/matplotib-finance-mplfinance-formatting-axes-of-chart-unsing-mpf-plot



In [ ]:
def plot_candlestick(df:pd.DataFrame, ax=None):
  if ax is None:
    fig, ax = plt.subplots(figsize=(16, 9))

  fplt.plot(
      df,
      type='candle',
      style='charles',
      ylabel='Price ($)',
      ax=ax,
      datetime_format=""
  );

  ax.grid(True)

  return ax



## Volume Profile : daily

In [ ]:
def get_one_day_from_df(df:pd.DataFrame, date:pd.Timestamp) -> pd.DataFrame:
  return df.loc[(df.index >= date) & (df.index < date + USBday())]



Histogram values of a Pandas Series https://stackoverflow.com/questions/13129618/histogram-values-of-a-pandas-series



Feature Request: Volume Profile on a chart with alpha argument #162 https://github.com/matplotlib/mplfinance/issues/162



In [ ]:
@functools.lru_cache()
def round(price_usd:float, resolution_usd:float=resolution_usd):
  assert 0 < resolution_usd
  return math.floor(price_usd / resolution_usd) * resolution_usd



In [ ]:
assert 100.119 < round(100.1234, resolution_usd=0.01)
assert 100.121 > round(100.1234, resolution_usd=0.01)



In [ ]:
def get_bins_min_max(min_close:float, max_close:float, resolution_usd:float=resolution_usd) -> np.ndarray:
  return np.arange(round(min_close, resolution_usd), max_close + (0.1 * resolution_usd), resolution_usd)



In [ ]:
def get_bins(ser:pd.Series, resolution_usd:float):
  return get_bins_min_max(ser.min(), ser.max(), resolution_usd)



In [ ]:
def calc_histogram_close(df:pd.DataFrame, bins:np.ndarray) -> Tuple[np.ndarray]:
  return np.histogram(df["Close"], bins=bins, weights=df["Volume"])



How to Analyze Volume Profiles With Python https://medium.com/swlh/how-to-analyze-volume-profiles-with-python-3166bb10ff24



In [ ]:
def estimate_kde(
    df_one_day:pd.DataFrame,
    ax=None,
    bins:np.ndarray=None,
    kde_factor:float=0.05
  ) -> Tuple[np.ndarray]:

  kde = ss.gaussian_kde(
      df_one_day["Close"],
      weights=df_one_day["Volume"],
      bw_method=kde_factor
  )

  kdy = kde(bins)

  return bins, kdy



In [ ]:
def close_kde_polygon(
    price:np.ndarray,
    density:np.ndarray
  ) -> Tuple[np.ndarray]:

  return (
      np.append(
          price,
          [price[-1], price[0]]
      ),
      np.append(
          density,
          [0.0, 0.0]
      ),
    )



In [ ]:
def plot_kde(df_one_day:pd.DataFrame, ax=None, bins=None, kde_factor:float=0.05, alpha:float=0.5):
  price, density = close_kde_polygon(
      *estimate_kde(
          df_one_day, ax, bins, kde_factor=kde_factor
      )
  )
  ax.fill(density, price, alpha=alpha)
  return ax



In [ ]:
def plot_vprofile(df_one_day:pd.DataFrame, ax=None, bins=None):
  width, y = calc_histogram_close(df_one_day, bins=bins)

  ax.set_xlim((0, 1.2*width.max()))
  ax.barh(y=y[:-1], width=width, height=resolution_usd, alpha=0.5);

  return ax



In [ ]:
def plot_candlestick_vprofile(df_one_day:pd.DataFrame, ax=None, bins=None):
  ax = plot_candlestick(df_one_day, ax=ax)

  ax2 = ax.twiny()
  try:
    plot_kde(df_one_day, ax2, bins)
  except ValueError:
    plot_vprofile(df_one_day, ax2, bins)

  return ax, ax2



In [ ]:
@functools.lru_cache()
def get_dates_tuple_start_end(start:pd.Timestamp, end:pd.Timestamp, tz=tz()):
  return tuple(
      pd.date_range(
          start, end,
          tz=tz,
          freq=USBday(),
      )
  )



In [ ]:
def get_dates_tuple(df:pd.DataFrame, tz=tz()) -> Tuple[pd.Timestamp]:
  return get_dates_tuple_start_end(df.index.min(), df.index.max(), tz=tz)



Creating adjacent subplots https://matplotlib.org/stable/gallery/subplots_axes_and_figures/ganged_plots.html



Show tick labels when sharing an axis in matplotlib https://stackoverflow.com/questions/29266966/show-tick-labels-when-sharing-an-axis-in-matplotlib



In [ ]:
def plot_daily_profile(ticker:str, n_days:int, end_date:pd.Timestamp=today(), interval:str=interval, resolution_usd:float=resolution_usd, figsize:Tuple[int]=(16,9)):
  start_date = get_start_date(n_days=n_days, end_date=end_date)
  dates = get_dates_tuple_start_end(start_date, end_date)

  fig, axs = plt.subplots(nrows=1, ncols=len(dates), figsize=figsize, sharey=True)
  fig.subplots_adjust(wspace=0)
  fig.suptitle(ticker)

  min_list = []
  max_list = []
  df_list = []

  for date in dates:
    df_date = get_df_one_day(ticker=ticker, date=date, interval=interval)
    df_list.append(df_date)
    min_list.append(df_date["Close"].min())
    max_list.append(df_date["Close"].max())

  bins_historical = get_bins_min_max(min(min_list), max(max_list), resolution_usd)

  ax_collection = []

  for date, ax, df_date in zip(dates, axs, df_list):
    ax_collection.append(
      plot_candlestick_vprofile(df_date, ax=ax, bins=bins_historical)
    )

    min_list.append(df_date["Close"].min())
    max_list.append(df_date["Close"].max())

  ax_collection[0][0].yaxis.set_tick_params(labelleft=True)
  ax_collection[-1][0].yaxis.set_tick_params(labelright=True)

  return ax_collection



## Running the code


In [ ]:
ticker = "SPY"



In [ ]:
plot_daily_profile(ticker, n_days=10, figsize=(30,10));



### Multiple tickers



In [ ]:
tickers = ("QQQ", "FNGS", "BLFS", "TSLA", "NVDA", "MSFT", "O", "VNQ", "BLK", "IYF", "AEHR")



In [ ]:
for ticker in tickers:
  plot_daily_profile(ticker, n_days=10, figsize=(30,10));

